In [1]:
import pandas as pd
import math

## This notebook is used to analyze the absolute time intervales between commits for each project to get a sence of the difference in time between snapshots

In [2]:
sql=pd.read_csv("SQL_project_versions.csv")
nsql=pd.read_csv("NoSQL_Project_versions.csv")

In [3]:
sql.head(1)

,project,version,sha,date
0,quran_quran_android,1,456d81b13031331a028f57a2e52c1cb0efcc74e7,2011-08-02T10:13:54+00:00


In [7]:
sql["date"]=pd.to_datetime(sql["date"])

In [10]:
nsql["date"]=pd.to_datetime(nsql["date"])

In [20]:
sql['delta'] = sql['date']-sql['date'].shift()
sql=sql.drop(columns=['delta'])

In [21]:
#split the df by project
projects= list(set(sql["project"].tolist()))

In [22]:
#split the dataframe by projects
sql_dfs=[]
for index, p in enumerate(projects):
    df= sql[sql["project"]==p]
    sql_dfs.append(df)

In [34]:
#now compute the delta for each frame
for d in sql_dfs:
    d['delta']=(d['date']-d['date'].shift()).dt.days
sql_computed=pd.concat(sql_dfs)
sql_computed=sql_computed[~sql_computed["delta"].isnull()] #remove the first versions with 0 delata

<ipython-input-34-0d62105af5a3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['delta']=(d['date']-d['date'].shift()).dt.days


In [35]:
#repeat for NoSQL
projects= list(set(nsql["project"].tolist()))
#split the dataframe by projects
nsql_dfs=[]
for index, p in enumerate(projects):
    df= nsql[nsql["project"]==p]
    nsql_dfs.append(df)
#now compute the delta for each frame
for d in nsql_dfs:
    d['delta']=(d['date']-d['date'].shift()).dt.days #convert the difference to days
nsql_computed=pd.concat(nsql_dfs)
nsql_computed=nsql_computed[~nsql_computed["delta"].isnull()] #remove the first versions with 0 delata

<ipython-input-35-4ee19f0faa06>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['delta']=(d['date']-d['date'].shift()).dt.days


In [36]:
nsql_computed.head()

,project,version,sha,date,delta
32,RyanSusana_elepy,2,e91802796253027524f1629e8b511504b124e61a,2019-01-18 11:00:01+00:00,268.0
33,RyanSusana_elepy,3,e335177c87920864402a77b56b71f7dda2ce0f81,2019-07-17 10:27:32+00:00,179.0
34,RyanSusana_elepy,4,e46a9577f74dd6a09c9df7acd659c9aac5e2e55d,2020-05-21 23:00:01+00:00,309.0
68,hazelcast_hazelcast-jet,2,1056235cd74f9742084b7d417d22c6427d1608b8,2017-02-06 18:40:34+00:00,161.0
69,hazelcast_hazelcast-jet,3,d52982f0e8274a25f824aecd132c1275cf2efe51,2018-05-11 12:46:46+00:00,458.0


                                          mean  count          std
project                                                           
AdrienPoupa_VinylMusicPlayer        582.666667      3   384.173572
AndlyticsProject_andlytics          889.666667      3  1284.580217
AntennaPod_AntennaPod               234.846154     13   138.938143
AppLozic_Applozic-Android-SDK       528.666667      3   256.480669
AxonFramework_AxonFramework         316.454545     11   303.794458
...                                        ...    ...          ...
wordpress-mobile_WordPress-Android   40.549451     91    82.623277
wso2_carbon-apimgt                   39.750000     48    41.985053
wso2_product-apim                   136.642857     14   110.242814
xipki_xipki                         185.416667     12   135.585906
yukuku_androidbible                 509.714286      7   502.347063

[84 rows x 3 columns]


In [70]:
#Taken from the stackoverflow answer https://stackoverflow.com/a/53522680
#compute the mean and confidence interval
stats = sql_computed.groupby(['project'])['delta'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.95*s/math.sqrt(c))
    ci95_lo.append(m - 1.95*s/math.sqrt(c))
stats['ci95_low'] = ci95_lo
stats['ci95_high'] = ci95_hi
sql_stats=stats
#NoSQL
#Taken from the stackoverflow answer https://stackoverflow.com/a/53522680
#compute the mean and confidence interval
stats = nsql_computed.groupby(['project'])['delta'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.95*s/math.sqrt(c))
    ci95_lo.append(m - 1.95*s/math.sqrt(c))
stats['ci95_low'] = ci95_lo
stats['ci95_high'] = ci95_hi
nsql_stats=stats

In [71]:
sql_stats["project"]=sql_stats.index
sql_stats=sql_stats.reset_index(drop=True)
sql_stats=sql_stats.reindex(columns=['project', 'count','mean', 'std', 'ci95_low', 'ci95_high'])
sql_stats.head()
sql_stats.to_csv("SQL_projects_commit_time_stat.csv", index=False)

In [72]:
nsql_stats["project"]=nsql_stats.index
nsql_stats=nsql_stats.reset_index(drop=True)
nsql_stats=nsql_stats.reindex(columns=['project', 'count','mean', 'std', 'ci95_low', 'ci95_high'])
nsql_stats.head()
nsql_stats.to_csv("NSQL_projects_commit_time_stat.csv", index=False)

In [17]:
#explore the result dataset
sql= pd.read_csv("SQL_projects_commit_time_stat.csv")
nosql=pd.read_csv("NSQL_projects_commit_time_stat.csv")

In [18]:
len(sql.index)
sql.head()
nosql.head()

,Subject system,Snapshots,Mean,SD,95_CI_low,95_CI_high
0,Flipkart_foxtrot,5,440.40,247.40,224.65,656.15
1,Hurence_logisland,4,321.75,126.33,198.58,444.92
2,IHTSDO_snowstorm,3,445.33,186.16,235.75,654.91
3,OpenSextant_Xponents,3,830.67,225.25,577.07,1084.26
4,RyanSusana_elepy,3,252.00,66.46,177.18,326.82


In [22]:
sql["group"]="SQL"
nosql["group"]="NoSQL"
comb=pd.concat([sql,nosql])
comb.to_csv("combined_projects_commit_time_stat.csv", index=False)

In [16]:
sql.columns

Index(['Subject system', 'Snapshots', 'Mean', 'SD', '95_CI_low', '95_CI_high',
       'group'],
      dtype='object')

In [24]:
comb.groupby(by=["group"]).mean()

,Snapshots,Mean,SD,95_CI_low,95_CI_high
group,,,,,
NoSQL,3.736842,534.986842,291.844211,210.281579,859.693684
SQL,14.265060,423.210843,323.130964,108.111928,738.309277
